In [ ]:
%pip install mlxtend
%pip install openpyxl

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('G:/Mi unidad/Python/Association Rules/Online Retail.xlsx')
df.head()

In [ ]:
# Algunas descripciones tienen espacios que se eliminarán
# También eliminaremos las filas que no tienen números de factura y eliminaremos las transacciones de crédito (aquellas con números de factura que contienen C).

df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [ ]:
# Revisamos transacciones de compara de un país específico
df[df['Country'] =="France"].head()

In [ ]:
# Se consolidan los artículos en 1 transacción agrupando por factura / producto 

basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket.head()

In [ ]:
# Cualquier valor positivo se convierte en 1 y cualquier valor menor que 0 se establece en 0.
# Este paso completa la codificación Hot Encode de los datos y se elimina la columna POSTAGE ya que no se requiere explorar
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

basket_sets.head()

In [ ]:
# Se genera un conjunto de elementos frecuentes que tengan un soporte de al menos el 7%
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
# Generar las reglas con su correspondiente soporte, confianza y posibilidad de que suceda (lift)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules.head()

In [ ]:
# filtrar el dataframe con valores de lift mayores a (6) y una confianza alta (.8), ordenado por LIFT
rules[ (rules['lift'] >= 4) &
       (rules['confidence'] >= 0.6) ].sort_values(by=['lift'], ascending=False)

In [ ]:
# Analizar cómo varían las combinaciones según el país de compra

basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.03, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1.2)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.6)].sort_values(by=['lift'], ascending=False)